In [9]:
import numpy as np
import re
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from gensim.models import Word2Vec
from tensorflow.keras.utils import to_categorical
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

# Step 1: Import dataset
df = pd.read_csv('dataset/tweet.csv')

def clean_text(text):
    # Remove special characters, URLs, and numbers
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'\@\w+|\#', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    
    # Lowercase and tokenize
    text = text.lower().split()
    
    # Remove stopwords using Sastrawi (Indonesian stopword remover)
    stopword_factory = StopWordRemoverFactory()
    stop_words = set(stopword_factory.get_stop_words())
    
    text = [word for word in text if word not in stop_words]
    return text

df['cleaned_tweet'] = df['tweet']

# Step 3: Word2Vec - Training a Word2Vec model (Skipgram)
sentences = df['cleaned_tweet'].tolist()
w2v_model = Word2Vec(sentences, vector_size=100, window=5, sg=1, min_count=1)

# Step 4: Tokenization and Padding
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['cleaned_tweet'].apply(lambda x: ' '.join(x)))
sequences = tokenizer.texts_to_sequences(df['cleaned_tweet'].apply(lambda x: ' '.join(x)))

# Pad sequences to ensure uniform input size
max_sequence_length = max([len(x) for x in sequences])
X = pad_sequences(sequences, maxlen=max_sequence_length, padding='post')

# Convert the target labels into categorical format
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['sentimen'])
y = to_categorical(y)

# Step 5: Train, Validation, and Test Split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Step 6: LSTM Model Construction
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, 100))
for word, i in tokenizer.word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]

# Updated Model Architecture
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, 
                    output_dim=100, 
                    weights=[embedding_matrix], 
                    trainable=False))  # Set trainable=False to use pre-trained embeddings

# Increase LSTM units and use Bidirectional LSTM
model.add(Bidirectional(LSTM(128, return_sequences=False)))  # Increase LSTM units to 128
model.add(Dropout(0.4))
model.add(Dense(64, activation='relu'))  # Increase dense layer units to 64
model.add(Dropout(0.3))
model.add(Dense(y.shape[1], activation='softmax'))

optimizer = Adam(learning_rate=0.0001)  # Change the learning rate to a different value
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Step 8: Model Training without Early Stopping
history = model.fit(X_train, y_train, 
                    epochs=100,  # Keep the epochs as is
                    batch_size=32,  # Keep the batch size as is
                    validation_data=(X_val, y_val), 
                    verbose=2)

# Step 9: Model Evaluation
train_loss, train_accuracy = model.evaluate(X_train, y_train, verbose=0)
val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)

print(f"Train Accuracy: {train_accuracy * 100:.2f}%")
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Step 10: Save the LSTM model
model.save('lstm_sentiment_model_tuned.h5')

Epoch 1/100
40/40 - 21s - 520ms/step - accuracy: 0.3402 - loss: 1.1022 - val_accuracy: 0.3309 - val_loss: 1.1041
Epoch 2/100
40/40 - 17s - 426ms/step - accuracy: 0.3189 - loss: 1.1016 - val_accuracy: 0.3493 - val_loss: 1.1014
Epoch 3/100
40/40 - 18s - 446ms/step - accuracy: 0.3417 - loss: 1.0991 - val_accuracy: 0.2794 - val_loss: 1.1037
Epoch 4/100
40/40 - 18s - 440ms/step - accuracy: 0.3543 - loss: 1.0979 - val_accuracy: 0.3419 - val_loss: 1.0984
Epoch 5/100
40/40 - 18s - 449ms/step - accuracy: 0.3598 - loss: 1.0970 - val_accuracy: 0.3566 - val_loss: 1.0998
Epoch 6/100
40/40 - 18s - 440ms/step - accuracy: 0.3496 - loss: 1.0945 - val_accuracy: 0.3603 - val_loss: 1.1001
Epoch 7/100
40/40 - 19s - 470ms/step - accuracy: 0.3622 - loss: 1.0967 - val_accuracy: 0.3640 - val_loss: 1.1006
Epoch 8/100
40/40 - 18s - 441ms/step - accuracy: 0.3850 - loss: 1.0938 - val_accuracy: 0.3419 - val_loss: 1.0945
Epoch 9/100
40/40 - 22s - 561ms/step - accuracy: 0.3850 - loss: 1.0909 - val_accuracy: 0.3235 - 

Train Accuracy: 51.57%
Validation Accuracy: 50.74%
Test Accuracy: 53.48%
